# 8 Exceptional Control Flow

## 8.1 Exceptions
An _exception_ is an abrupt change in the control flow in response to some
change in the processor’s state. **Figure 8.1** shows the basic idea.

![](asset/ch8/1.png)

In the figure, the processor is executing some current instruction $I_{curr}$ when a significant change in the processor’s `state` occurs. The state is encoded in various bits and signals inside the processor. The change in state is known as an `event`.

The event might be directly related to the execution of the current instruction.
For example, a virtual memory page fault occurs, an arithmetic overflow occurs,
or an instruction attempts a divide by zero. On the other hand, the event might be
unrelated to the execution of the current instruction. For example, a system timer
goes off or an I/O request completes.

In any case, when the processor detects that the event has occurred, it makes
an indirect procedure call (the exception), through a jump table called an _exception table_, to an operating system subroutine (the _exception handler_) that is specifically designed to process this particular kind of event. When the exception handler finishes processing, one of three things happens, depending on the type of event that caused the exception:

1. The handler returns control to the current instruction $I_{curr}$ , the instruction that was executing when the event occurred.
2. The handler returns control to $I_{next}$, the instruction that would have executed next had the exception not occurred.
3. The handler aborts the interrupted program

### 8.1.1 Exception Handling
Each type of possible exception in a system is assigned a unique nonnegative
integer _exception number_. Some of these numbers are assigned by the designers
of the processor. Other numbers are assigned by the designers of the operating
system _kernel_ (the memory-resident part of the operating system).

Each type of possible exception in a system is assigned a unique nonnegative integer _exception number_. Some of these numbers are assigned by the designers of the processor. Other numbers are assigned by the designers of the operating system _kernel_ (**the memory-resident part of the operating system**). Examples of the former include divide by zero, page faults, memory access violations, breakpoints, and arithmetic overflows. Examples of the latter include system calls and signals from external I/O devices.

![](asset/ch8/2.png)

At system boot time (when the computer is reset or powered on), the operating system allocates and initializes a jump table called an _exception table_, so that entry $k$ contains the address of the handler for exception $k$. **Figure 8.2** shows the format of an exception table.

![](asset/ch8/3.png)

At run time (when the system is executing some program), the processor detects that an event has occurred and determines the corresponding exception number $k$. The processor then triggers the exception by making an indirect procedure call, through entry $k$ of the exception table, to the corresponding handler. **Figure 8.3** shows how the processor uses the exception table to form the address of the appropriate exception handler. The exception number is an index into the exception table, whose starting address is contained in a special CPU register called the _exception table base register_.

An exception is akin to a procedure call, but with some important differences:
* As with a procedure call, the processor pushes a return address on the stack before branching to the handler. However, depending on the class of exception, the return address is either the current instruction or the next instruction.
* The processor also pushes some additional processor state onto the stack that will be necessary to restart the interrupted program when the handler returns. For example, an x86-64 system pushes the EFLAGS register containing the current condition codes, among other things, onto the stack.
* When control is being transferred from a user program to the kernel, all of these items are pushed onto the kernel’s stack rather than onto the user’s stack.
* Exception handlers run in _kernel mode_, which means they have
complete access to all system resources.

Once the hardware triggers the exception, the rest of the work is done in software by the exception handler. After the handler has processed the event, it optionally returns to the interrupted program by executing a special “return from interrupt” instruction, which pops the appropriate state back into the processor’s control and data registers, restores the state to _user mode_ if the exception interrupted a user program, and then returns control to the interrupted program.

### 8.1.2 Classes of Exceptions

Exceptions can be divided into four classes: _interrupts, traps, faults, and aborts_. The table in **Figure 8.4** summarizes the attributes of these classes.

![](asset/ch8/4.png)

**Interrupts** 
_Interrupts_ occur _asynchronously_ as a result of signals from I/O devices that are external to the processor. Hardware interrupts are asynchronous in the sense that they are not caused by the execution of any particular instruction. Exception handlers for hardware interrupts are often called _interrupt handlers_.
